In [1]:
import warnings

import keras
import matplotlib.pyplot as plt
import pickle
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from fuzzywuzzy import fuzz, process
from keras.layers import Dense
from keras.models import Sequential, load_model
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from nltk.corpus import stopwords
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from xgboost.sklearn import XGBClassifier

stop = stopwords.words('english')
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.
C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

In [2]:
df_clean = pd.read_csv('./data/df_clean.csv', index_col=0)
df_test = pd.read_csv('./data/test.csv')

In [3]:
df_clean.head()

,query,product_title,product_description,median_relevance,relevance_variance,target,prod_clean,desc_clean,prod_fuzzy,desc_fuzzy,...,francisco,prom,clothes,case,bay,note,comforter,tick,control,toe
id,,,,,,,,,,,,,,,,,,,,,
1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000,1,accent pillow heart design - red/black,red satin accent pillow embroidered heart blac...,44,44,...,False,False,False,False,False,False,False,False,False,False
2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000,4,set battery operated multi led train christma...,set battery operated train christmas lights i...,85,50,...,False,False,False,False,False,False,False,False,False,False
4,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471,4,viewsonic pro dlp multimedia projector,none,100,50,...,False,False,False,False,False,False,False,False,False,False
5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000,4,concept housewares wr- solid-wood ceiling/wall...,"like silent sturdy tree, southern enterprises ...",100,56,...,False,False,False,False,False,False,False,False,False,False
7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471,2,wintergreen lighting christmas led light bulb ...,"wtgr features nickel base, , average hours, ac...",100,100,...,False,False,False,False,False,False,False,False,False,False


In [4]:
X = df_clean.loc[:, 'prod_fuzzy':]
y = df_clean['target']

In [5]:
tup_list = pickle.load( open('data/tup_list.pkl', "rb" ) )

In [6]:
df_test['prod_clean'] = df_test['product_title'].str.lower().apply(lambda x: ' '.join(
    [word for word in x.split() if word not in (stop)]))
df_test['prod_clean'] = df_test['prod_clean'].str.replace('\d+','')
df_test['desc_clean'] = df_test['product_description'].str.lower().fillna('none').apply(
    lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_test['desc_clean'] = df_test['desc_clean'].str.replace('\d+', '')

In [7]:
df_test['prod_fuzzy'] = [fuzz.partial_ratio(
    df_test['query'][x], df_test['prod_clean'][x]) for x in range(df_test.shape[0])]
df_test['desc_fuzzy'] = [fuzz.partial_ratio(
    df_test['query'][x], df_test['desc_clean'][x]) for x in range(df_test.shape[0])]
df_test['prod_desc'] = [fuzz.partial_ratio(df_test['prod_clean'][x], df_test['desc_clean'][x])
                   for x in range(df_test.shape[0])]

In [8]:
df_test['query_len'] = df_test['query'].str.count(' ')+1

In [9]:
for term in tup_list:
    df_test[term] = df_test['query'].str.contains(term)

In [10]:
df_test.head()

,id,query,product_title,product_description,prod_clean,desc_clean,prod_fuzzy,desc_fuzzy,prod_desc,query_len,...,francisco,prom,clothes,case,bay,note,comforter,tick,control,toe
0,3,electric griddle,Star-Max 48 in Electric Griddle,NaN,star-max electric griddle,none,100,25,25,2,...,False,False,False,False,False,False,False,False,False,False
1,6,phillips coffee maker,Philips SENSEO HD7810 WHITE Single Serve Pod C...,NaN,philips senseo hd white single serve pod coffe...,none,67,50,50,3,...,False,False,False,False,False,False,False,False,False,False
2,9,san francisco 49ers,2013 San Francisco 49ers Clock,A 2013 San Francisco 49ers clock is the ultima...,san francisco ers clock,san francisco ers clock ultimate way show tea...,89,89,100,3,...,True,False,False,False,False,False,False,False,False,False
3,11,aveeno shampoo,AVEENO 10.5FLOZ NRSH SHINE SH,"Water, Ammonium Lauryl Sulfate, Dimethicone, S...",aveeno .floz nrsh shine sh,"water, ammonium lauryl sulfate, dimethicone, s...",57,43,38,2,...,False,False,False,False,False,False,False,False,False,False
4,12,flea and tick control for dogs,Merial Frontline Plus Flea and Tick Control fo...,NaN,merial frontline plus flea tick control dogs p...,none,73,50,50,6,...,False,False,False,False,False,False,False,True,True,False


In [11]:
test_df= df_test.set_index('id')

In [12]:
test_X = df_test.loc[:, 'prod_fuzzy':]

In [13]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X, y)
dum_pred = dummy.predict(test_X)

In [14]:
dum_df = pd.DataFrame(dum_pred, index=test_df.index, columns=['prediction'])

In [15]:
dum_df.to_csv('./data/dummy_sumbit.csv')

In [16]:
scaler= StandardScaler()
train_x = scaler.fit_transform(X)
test_x = scaler.transform(test_X)

In [17]:
lr = LogisticRegression(solver='lbfgs', class_weight='balanced')
lr.fit(train_x, y)
pred_lr = lr.predict(test_x)

In [18]:
lr_df = pd.DataFrame(pred_lr, index=test_df.index, columns=['prediction'])

In [19]:
lr_df.to_csv('./data/lr_submit.csv')

In [20]:
svc = SVC(kernel='rbf', gamma=1, C=1,
              decision_function_shape='ovo', class_weight='balanced').fit(train_x, y)
pred_rbf = svc.predict(test_x)

In [21]:
svc_df = pd.DataFrame(pred_rbf, index=test_df.index, columns=['prediction'])

In [22]:
svc_df.to_csv('./data/svc_submit.csv')

In [23]:
et = ExtraTreesClassifier(n_estimators=1000, min_samples_split=15).fit(train_x,y)
pred_et = et.predict(test_x)
et_df = pd.DataFrame(pred_et, index=test_df.index, columns=['prediction'])

In [24]:
et_df.to_csv('./data/et_submit.csv')

In [25]:
xgb = XGBClassifier(learning_rate=0.1,
                    n_estimators=1000,
                    max_depth=5,
                    min_child_weight=1,
                    gamma=0,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective='multi:softmax',
                    nthread=4,
                    num_class=9,
                    seed=27)
xgb.fit(train_x, y)
pred_xgb = xgb.predict(test_x)
xgb_df = pd.DataFrame(pred_xgb, index=test_df.index, columns=['prediction'])

In [26]:
xgb_df.to_csv('./data/xgb_submit.csv')